In [1]:
import sys
import os
import ast
import re
import glob
import json
import inspect
import math
import quantities

from functools import reduce # forward compatibility
import operator

import gc
import numpy
import scipy

import pylab
import matplotlib
import matplotlib.pyplot as plt

import quantities as qt

from parameters import ParameterSet

import mozaik
import mozaik.controller
from mozaik.visualization.plotting import *
from mozaik.analysis.technical import NeuronAnnotationsToPerNeuronValues
from mozaik.analysis.analysis import *
from mozaik.analysis.vision import *
from mozaik.storage.queries import *
from mozaik.storage.datastore import PickledDataStore

from mozaik.tools.mozaik_parametrized import colapse

import neo


In [11]:
def select_ids_by_position(positions, sheet_ids, radius=[0,0], box=[], reverse=False, origin=[[0.],[0.],[0.]]):
	selected_ids = []
	distances = []
	min_radius = radius[0]
	max_radius = radius[1]

	for i in sheet_ids:
		a = numpy.array((positions[0][i],positions[1][i],positions[2][i]))
		# print a

		if len(box)>1:
			# Ex box: [ [-.3,.0], [.3,-.4] ]
			# Ex a: [ [-0.10769224], [ 0.16841423], [ 0. ] ]
			# print box[0][0], a[0], box[1][0], "      ", box[0][1], a[1], box[1][1]
			if a[0]>=box[0][0] and a[0]<=box[1][0] and a[1]>=box[0][1] and a[1]<=box[1][1]:
				# print a[0], "   ", a[1]
				selected_ids.append(i[0])
				distances.append(0.0)
		else:
			# print a # from origin
			l = numpy.linalg.norm(origin-a)

			# print "distance",l
			# print abs(l),">",min_radius,"and", abs(l), "<", max_radius
			if abs(l)>min_radius and abs(l)<max_radius:
				# print abs(l),">",min_radius,"and", abs(l), "<", max_radius, "TAKEN"
				# print i
				selected_ids.append(i[0])
				distances.append(l)

	# sort by distance
	# print len(selected_ids)
	# print distances
	return [x for (y,x) in sorted(zip(distances,selected_ids), key=lambda pair:pair[0], reverse=reverse)]




def LFP(data_store, sheet, folder, stimulus, parameter, tip=[.0,.0,.0], sigma=0.300, ylim=[0.,-1.], addon="", color='black' ):
	import matplotlib as ml
	import quantities as pq
	#print inspect.stack()[0][3]
	print "folder: ",folder
	print "sheet: ",sheet

	segs = sorted( 
		param_filter_query(data_store, st_name=stimulus, sheet_name=sheet).get_segments(), 
		key = lambda x : getattr(MozaikParametrized.idd(x.annotations['stimulus']), parameter) 
	)
	print len(segs)
	# for s in segs:
	# 	s.load_full()

	ids = param_filter_query(data_store, sheet_name=sheet, st_name=stimulus).get_segments()[0].get_stored_vm_ids()
	if ids == None or len(ids)<1:
		print "No Vm recorded.\n"
		return

	print "Recorded neurons:", len(ids), ids
	# 900 neurons over 6000 micrometers, 200 micrometers interval

	sheet_indexes = data_store.get_sheet_indexes(sheet_name=sheet, neuron_ids=ids)

	positions = data_store.get_neuron_postions()[sheet]
	print positions.shape # all 10800

	# take the positions of the ids
	ids_positions = numpy.transpose(positions)[sheet_indexes,:]
	print ids_positions.shape
	print ids_positions

	# Pre-compute distances from the LFP tip
	distances = []
	for i in range(len(ids)):
		distances.append( numpy.linalg.norm( numpy.array(ids_positions[i][0]) - numpy.array(tip) ) ) 
	distances = numpy.array(distances)
	print "distances:", len(distances), distances

	# ##############################
	# LFP
	# tip = [[x],[y],[.0]]
	# For each recorded cell:
	# Gaussianly weight it by its distance from tip
	# produce the currents
	# Divide the whole by the norm factor (area): 4 * numpy.pi * sigma

	# 95% of the LFP signal is a result of all exc and inh cells conductances from 250um radius from the tip of the electrode (Katzner et al. 2009).
	# Mostly excitatory neurons are relevant for the LFP (because of their geometry) Bartos
	# Therefore we include all recorded cells but account for the distance-dependent contribution weighting currents /r^2
	# We assume that the electrode has been placed in the cortical coordinates <tip>
	# Given that the current V1 orientation map has a pixel for each 100 um, a reasonable way to look at a neighborhood is in a radius of 300 um

	print "LFP electrode tip location (x,y) in degrees:", tip

	# Gather vm and conductances
	segs = sorted( 
		param_filter_query(data_store, st_name=stimulus, sheet_name=sheet).get_segments(), 
		key = lambda x : getattr(MozaikParametrized.idd(x.annotations['stimulus']), parameter) 
	)
	ticks = set([])
	for x in segs:
		ticks.add( getattr(MozaikParametrized.idd(x.annotations['stimulus']), parameter) )
	ticks = sorted(ticks)
	num_ticks = len( ticks )
	print ticks
	trials = len(segs) / num_ticks
	print "trials:",trials

	pop_vm = []
	pop_gsyn_e = []
	pop_gsyn_i = []
	print "Number of neurons considering" , len(ids)
	for n,idd in enumerate(ids):
		print "idd", idd
		full_vm = [s.get_vm(idd) for s in segs] # all segments
		full_gsyn_es = [s.get_esyn(idd) for s in segs]
		full_gsyn_is = [s.get_isyn(idd) for s in segs]
		print "len full_gsyn_e", len(full_gsyn_es) # segments = stimuli * trials
		print "shape gsyn_e[0]", full_gsyn_es[0].shape # stimulus lenght
		# mean input over trials
		mean_full_vm = numpy.zeros((num_ticks, full_vm[0].shape[0])) # init
		mean_full_gsyn_e = numpy.zeros((num_ticks, full_gsyn_es[0].shape[0])) # init
		mean_full_gsyn_i = numpy.zeros((num_ticks, full_gsyn_es[0].shape[0]))
		# print "shape mean_full_gsyn_e/i", mean_full_gsyn_e.shape
		sampling_period = full_gsyn_es[0].sampling_period
		t_stop = float(full_gsyn_es[0].t_stop - sampling_period) # 200.0
		t_start = float(full_gsyn_es[0].t_start)
		time_axis = numpy.arange(0, len(full_gsyn_es[0]), 1) / float(len(full_gsyn_es[0])) * abs(t_start-t_stop) + t_start
		# sum by size
		t = 0
		for v,e,i in zip(full_vm, full_gsyn_es, full_gsyn_is):
			s = int(t/trials)
			v = v.rescale(quantities.mV) 
			e = e.rescale(mozaik.tools.units.nS) # NEST is in nS, PyNN is in uS
			i = i.rescale(mozaik.tools.units.nS) # NEST is in nS, PyNN is in uS
			mean_full_vm[s] = mean_full_vm[s] + numpy.array(v.tolist())
			mean_full_gsyn_e[s] = mean_full_gsyn_e[s] + numpy.array(e.tolist())
			mean_full_gsyn_i[s] = mean_full_gsyn_i[s] + numpy.array(i.tolist())
			t = t+1

		# average by trials
		for st in range(num_ticks):
			mean_full_vm[st] = mean_full_vm[st] / trials
			mean_full_gsyn_e[st] = mean_full_gsyn_e[st] / trials
			mean_full_gsyn_i[st] = mean_full_gsyn_i[st] / trials

		pop_vm.append(mean_full_vm)
		pop_gsyn_e.append(mean_full_gsyn_e)
		pop_gsyn_i.append(mean_full_gsyn_i)

	pop_v = numpy.array(pop_vm)
	pop_e = numpy.array(pop_gsyn_e)
	pop_i = numpy.array(pop_gsyn_i)

	# Produce the current for each cell for this time interval, with the Ohm law:
	# I = ge(V-Ee) + gi(V+Ei)
	# where 
	# Ee is the equilibrium for exc, which is 0.0
	# Ei is the equilibrium for inh, which is -80.0
	i = pop_e*(pop_v-0.0) + pop_i*(pop_v-80.0)
	# i = pop_e*(pop_v-0.0) + 0.3*pop_i*(pop_v-80.0)
	# i = pop_e*(pop_v-0.0) # only exc
	# the LFP is the result of cells' currents divided by the distance
	print numpy.shape(distances)
	print numpy.shape(i)
	i = i#/distances
	sum_i = numpy.sum(i, axis=0 )
	lfp = sum_i/(4*numpy.pi*sigma) #
	lfp /= 1000. # from milli to micro
	print "LFP:", lfp.shape, lfp.mean(), lfp.min(), lfp.max()
	# print lfp
	# lfp = np.convolve(lfp, np.ones((10,))/10, mode='valid') # moving avg or running mean implemented as a convolution over steps of 10, divided by 10
	# lfp = np.convolve(lfp, np.ones((10,))/10, mode='valid') # moving avg or running mean implemented as a convolution over steps of 10, divided by 10

	#plot the LFP for each stimulus
	for s in range(num_ticks):
		# for each stimulus plot the average conductance per cell over time
		matplotlib.rcParams.update({'font.size':22})
		fig,ax = plt.subplots()

		ax.plot( range(0,len(lfp[s])), lfp[s], color=color, linewidth=3 )

		# ax.set_ylim([lfp.min(), lfp.max()])
		ax.set_xlim([0,150])
		ax.set_ylim(ylim)
		ax.set_ylabel( "LFP (uV)" )
		ax.set_xlabel( "Time (us)" )
		ax.spines['right'].set_visible(False)
		ax.spines['top'].set_visible(False)
		ax.xaxis.set_ticks_position('bottom')
		ax.xaxis.set_ticks(ticks, ticks)
		ax.yaxis.set_ticks_position('left')

		# text
		plt.tight_layout()
		plt.savefig( folder+"/TimecourseLFP_"+"_"+parameter+"_"+str(ticks[s])+"_"+addon+".svg", dpi=200, transparent=True )
		fig.clf()
		plt.close()
		# garbage
		gc.collect()

In [3]:
data_store = PickledDataStore(load=True,parameters=ParameterSet({'root_directory':'/home/antolikjan/remote/UNIC/dev/pkg/mozaik/mozaik/contrib/New/20190912-113106[param_FINAL.defaults]CombinationParamSearch{sheets.l4_cortex_exc.K:[1000]}/SelfSustainedPushPull_ParameterSearch_____sheets.l4_cortex_exc.K:1000','store_stimuli': False}),replace=True)


In [15]:
# [0.14999999999999991,0.15000000000000002]
dsv = param_filter_query(data_store,st_name='FlashedInterruptedBar',st_y=0.15)    
dsv.print_content()
print data_store.get_segments()[160].annotations

INFO:Mozaik:DSV info:
INFO:Mozaik:   Number of recordings: 600
INFO:Mozaik:     FlashedInterruptedBar : 600
INFO:Mozaik:   Number of ADS: 30
INFO:Mozaik:     PerNeuronValue : 30


{'stimulus': '{"module_path" :"mozaik.stimuli.vision.topographica_based","background_luminance":50.0, "density":10.0, "direct_stimulation_name":\'None\', "disalignment":0, "duration":2002, "flash_duration":2002, "frame_duration":7, "gap_length":0.6, "length":20, "location_x":0.0, "location_y":0.0, "name":\'FlashedInterruptedBar\', "orientation":0, "relative_luminance":1.0, "size_x":11.0, "size_y":11.0, "trial":0, "width":0.3125, "x":4.5924254968025736e-17, "y":0.74999999999999989}', 'sheet_name': 'V1_Inh_L4'}


In [12]:
f = '/home/antolikjan/remote/UNIC/dev/pkg/mozaik/mozaik/contrib/New/20190912-113106[param_FINAL.defaults]CombinationParamSearch{sheets.l4_cortex_exc.K:[1000]}/SelfSustainedPushPull_ParameterSearch_____sheets.l4_cortex_exc.K:1000'

dsv = param_filter_query(data_store,st_name='DriftingSinusoidalGratingRing')    
dsv.print_content()

s = 'V1_Exc_L4'
addon = 'L4'
color = 'r'

LFP(
    data_store=dsv,
    sheet=s, 
    folder=f, 
    stimulus='DriftingSinusoidalGratingRing',
    parameter="inner_appareture_radius",
    tip = [.0,.0,.0], # CENTER box in which to search a centroid for LFP to measure the conductance effect in lfp_sheet
    # tip_box = [[0.,0.],[0.,0.]], # box in which to search a centroid for LFP to measure the conductance effect in lfp_sheet
    # tip_box = [[0.,0.],[0.,0.]], # box in which to search a centroid for LFP to measure the conductance effect in lfp_sheet
    sigma = 0.4, #   mm, since 1mm = 1deg in this cortical space (spikes from the center of sheet)
    # sigma = 0.1, # [0.1, 0.01] # Dobiszewski_et_al2012.pdf
    # sigma = 0.25, # 0.25 Bartos et al.
    ylim=None,
    addon = addon + "_center",
    color = color,
)

s = 'V1_Exc_L2/3'
addon = 'L23'
color = 'r'

LFP(
    data_store=dsv,
    sheet=s, 
    folder=f, 
    stimulus='DriftingSinusoidalGratingRing',
    parameter="inner_appareture_radius",
    tip = [.0,.0,.0], # CENTER box in which to search a centroid for LFP to measure the conductance effect in lfp_sheet
    # tip_box = [[0.,0.],[0.,0.]], # box in which to search a centroid for LFP to measure the conductance effect in lfp_sheet
    # tip_box = [[0.,0.],[0.,0.]], # box in which to search a centroid for LFP to measure the conductance effect in lfp_sheet
    sigma = 0.4, #   mm, since 1mm = 1deg in this cortical space (spikes from the center of sheet)
    # sigma = 0.1, # [0.1, 0.01] # Dobiszewski_et_al2012.pdf
    # sigma = 0.25, # 0.25 Bartos et al.
    ylim=None,
    addon = addon + "_center",
    color = color,
)

INFO:Mozaik:DSV info:
INFO:Mozaik:   Number of recordings: 1440
INFO:Mozaik:     DriftingSinusoidalGratingRing : 1440
INFO:Mozaik:   Number of ADS: 240
INFO:Mozaik:     PerNeuronValue : 240


folder:  /home/antolikjan/remote/UNIC/dev/pkg/mozaik/mozaik/contrib/New/20190912-113106[param_FINAL.defaults]CombinationParamSearch{sheets.l4_cortex_exc.K:[1000]}/SelfSustainedPushPull_ParameterSearch_____sheets.l4_cortex_exc.K:1000
sheet:  V1_Exc_L4
120
Recorded neurons: 225 [16064 25600 31936 31873 32065 32705 34241 37313 25346 33666 18755 22659
 25859 35587 17924 29636 29764 37316 19077 22341 32133 37573 16390 23494
 26950 28230 33222 37126 37318 31943 20168 25160 33864 23625 25865 28233
 29001 17226 23691 31435 23692 24012 26060 19917 36301 24270 25614 25934
 29454 36686 38094 38606 27087 32527 35343 21264 35344 28945 31569 33873
 35217 37009 17170 18706 20882 29906 30098 35282 28307 31891 32019 19732
 31892 24085 28309 30549 17302 19734 26839 30743 38039 17241 20633 28057
 28249 30617 37657 21914 30874 35418 38490 19739 25883 26779 36059 17884
 29532 17501 29085 33054 19551 26143 27167 30239 35935 16608 26912 29024
 31712 34208 36064 27425 29217 30305 36194 20451 20708 25892 30244

[0.0, 0.2727272727272727, 0.5454545454545454, 0.8181818181818181, 1.0909090909090908, 1.3636363636363635, 1.6363636363636362, 1.909090909090909, 2.1818181818181817, 2.454545454545454, 2.727272727272727, 3.0]
trials: 10
Number of neurons considering 225
idd 16064
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 25600
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 31936
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 31873
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 32065
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 32705
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 34241
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 37313
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 25346
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 33666
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 18755
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 22659
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 25859
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 35587
len full_gsyn_e 120
shape gsyn_e[0]

idd 23916
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 24172
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 38444
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 15405
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 32429
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 15022
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 26030
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 32494
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 34606
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 37166
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 22639
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 23791
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 29295
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 29551
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 34415
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 36015
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 15856
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 21808
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 25328
len full_gsyn_e 12

[0.0, 0.2727272727272727, 0.5454545454545454, 0.8181818181818181, 1.0909090909090908, 1.3636363636363635, 1.6363636363636362, 1.909090909090909, 2.1818181818181817, 2.454545454545454, 2.727272727272727, 3.0]
trials: 10
Number of neurons considering 225
idd 61440
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 45761
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 46465
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 47681
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 51010
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 54018
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 57090
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 62660
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 66564
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 67012
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 67332
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 67972
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 50437
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 54085
len full_gsyn_e 120
shape gsyn_e[0]

idd 50412
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 56748
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 61549
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 65837
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 48238
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 63342
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 59055
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 61039
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 63279
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 65071
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 67631
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 48496
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 49200
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 53872
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 60400
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 60912
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 64944
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 46897
len full_gsyn_e 120
shape gsyn_e[0] (2003,)
idd 51761
len full_gsyn_e 12